In [1]:
#The following packages were requried for install to complete this sentiment analysis 
#pip install textblob
#pip install wordcloud

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from textblob import TextBlob
from itertools import islice 
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
nltk.download('stopwords')
import wordcloud
from wordcloud import WordCloud, STOPWORDS

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jenna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Filter Tweet data for english tweets for the purpose of analyzing sentiment 
#tweets=pd.read_csv('TweetData.csv', engine='python')
tweets=pd.read_csv('TweetData-sample.csv', engine='python') #For efficiency purposes we will run the analysis on a sample 
english=tweets[tweets['lang'] == 'en']
english.columns

Index(['status_id', 'user_id', 'created_at', 'screen_name', 'text', 'source',
       'reply_to_status_id', 'reply_to_user_id', 'reply_to_screen_name',
       'is_quote', 'is_retweet', 'favourites_count', 'retweet_count',
       'country_code', 'place_full_name', 'place_type', 'followers_count',
       'friends_count', 'account_lang', 'account_created_at', 'verified',
       'lang', 'Date'],
      dtype='object')

In [4]:
#Segmented columns to be appended back in later 
append=english[['status_id', 'created_at', 'favourites_count','retweet_count','place_full_name','verified']]
append.head()

,status_id,created_at,favourites_count,retweet_count,place_full_name,verified
0,1.244050e+18,2020-03-29T00:00:16Z,1256.0,0.0,"Franklin, TN",False
1,1.244050e+18,2020-03-29T00:00:29Z,1284.0,0.0,"Los Angeles, CA",False
3,1.244050e+18,2020-03-29T00:00:39Z,14874.0,0.0,"Manhattan, NY",False
4,1.244050e+18,2020-03-29T00:00:42Z,4721.0,2.0,"Alexandria, LA",False
5,1.244050e+18,2020-03-29T00:01:13Z,178.0,0.0,"New Canaan, CT",False


In [5]:
#Formated text data to remove punctuation, special characters, capitalization, and stop wordds
english['text'] = english['text'].str.lower()
english['text'] = english['text'].str.replace('@[A-Za-z0-9]+', '')
english['text'] = english['text'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
english['text'] = english['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
english.head()

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang,Date
0,1.244050e+18,9.900702e+08,2020-03-29T00:00:16Z,realJohn_Stiles,flu seems fishy covid_19 numbers donât add,Twitter for iPad,1.243960e+18,50772918.0,ClayTravis,False,...,US,"Franklin, TN",city,375.0,762.0,NaN,2012-12-05T03:00:33Z,False,en,3/29/2020
1,1.244050e+18,2.854840e+09,2020-03-29T00:00:29Z,brightpinkinLA,omg ozark ending die thanks distraction covid1...,Twitter for iPhone,NaN,16573941.0,netflix,False,...,US,"Los Angeles, CA",city,187.0,1603.0,NaN,2014-10-13T21:49:00Z,False,en,3/29/2020
3,1.244050e+18,9.189890e+17,2020-03-29T00:00:39Z,forourchildern,dont think gd leadership across states see gov...,Twitter for Android,NaN,NaN,NaN,False,...,US,"Manhattan, NY",city,441.0,398.0,NaN,2017-10-13T23:55:14Z,False,en,3/29/2020
4,1.244050e+18,3.337320e+07,2020-03-29T00:00:42Z,cwhitcher1791,hospital engineering department helped create ...,Twitter for iPhone,NaN,NaN,NaN,False,...,US,"Alexandria, LA",city,365.0,777.0,NaN,2009-04-20T01:42:20Z,False,en,3/29/2020
5,1.244050e+18,8.612540e+17,2020-03-29T00:01:13Z,lifeakua,best make smile terotheotter pics keep full he...,Twitter for iPad,NaN,NaN,NaN,False,...,US,"New Canaan, CT",city,2956.0,472.0,NaN,2017-05-07T16:20:10Z,False,en,3/29/2020


In [6]:
#Created an empty Dataframe to load sentiment analysis into 
COLS = ['date','text', 'sentiment','subjectivity','polarity','status_id']
df = pd.DataFrame(columns=COLS)

In [7]:
#Sentiment Analysis to apply text blob Polarity and Sentiment scores to each cleaned tweet 
for index, row in islice(english.iterrows(), 0, None):
    
    new_entry = []
    text_lower=row['text']
    blob = TextBlob(text_lower)
    sentiment = blob.sentiment 
 
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity
 
    new_entry += [row['Date'],text_lower,sentiment,subjectivity,polarity,row['status_id']]
        
    sentimet_df = pd.DataFrame([new_entry], columns=COLS)
    df = df.append(sentimet_df, ignore_index=True)
    

df.head()

,date,text,sentiment,subjectivity,polarity,status_id
0,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0000,0.000000,1.244050e+18
1,3/29/2020,omg ozark ending die thanks distraction covid1...,"(0.2, 0.2)",0.2000,0.200000,1.244050e+18
2,3/29/2020,dont think gd leadership across states see gov...,"(-0.525, 0.7)",0.7000,-0.525000,1.244050e+18
3,3/29/2020,hospital engineering department helped create ...,"(-0.15, 0.5125)",0.5125,-0.150000,1.244050e+18
4,3/29/2020,best make smile terotheotter pics keep full he...,"(0.3416666666666666, 0.325)",0.3250,0.341667,1.244050e+18


In [8]:
#Append back in data previously segmented out based on stats_id 
final=pd.merge(df, append, left_on ='status_id', right_on='status_id', how="outer")

In [9]:
final.head()

,date,text,sentiment,subjectivity,polarity,status_id,created_at,favourites_count,retweet_count,place_full_name,verified
0,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0,0.0,1.244050e+18,2020-03-29T00:00:16Z,1256.0,0.0,"Franklin, TN",False
1,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0,0.0,1.244050e+18,2020-03-29T00:00:29Z,1284.0,0.0,"Los Angeles, CA",False
2,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0,0.0,1.244050e+18,2020-03-29T00:00:39Z,14874.0,0.0,"Manhattan, NY",False
3,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0,0.0,1.244050e+18,2020-03-29T00:00:42Z,4721.0,2.0,"Alexandria, LA",False
4,3/29/2020,flu seems fishy covid_19 numbers donât add,"(0.0, 0.0)",0.0,0.0,1.244050e+18,2020-03-29T00:01:13Z,178.0,0.0,"New Canaan, CT",False


In [10]:
#Code to write out final dataframe to a file to be used in other notebooks 

final_COLS=final.columns
#final.to_csv('Tweet_Text_Sentiment_data.csv', mode='w', columns=final_COLS, index=False, encoding='utf-8')

In [11]:
#Sources for developing sentiment analysis

#https://data-science-blog.com/en/blog/2018/11/04/sentiment-analysis-using-python/
#https://planspace.org/20150607-textblob_sentiment/